# Import Libraries

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency

# Load Dataset

In [ ]:
df = pd.read_csv("./Train.csv", index_col="ID")
df.head()

In [ ]:
df.rename(columns={'Reached.on.Time_Y.N': 'Arrived_Late'}, inplace=True)

In [ ]:
df.info()

### Cek Missing Values 

In [ ]:
df.isnull().any()

**Karena pada dataset kita ini tidak ada Missing Values, maka nanti saat tahap Data Pre-Processing, kita tidak perlu Handle Missing Values.**

### Cek Duplicated Data

In [ ]:
df.duplicated().any()

**Karena pada dataset kita ini tidak ada Data Duplikat, maka nanti saat tahap Data Pre-Processing, kita tidak perlu Handle Duplicated Data.**

# Descriptive Statistics

In [ ]:
cats_df = df.select_dtypes(include='object')
nums_df = df.select_dtypes(include='int')

cats = cats_df.columns
nums = nums_df.columns

In [ ]:
df[nums].describe().T

- Rata-rata Customer care menerima 4.05 panggilan dari seorang customer dan mediannya 4, berdasarkan hal tersebut dapat disimpulkan kolom ini memiliki persebaran normal.

- Customer rating memiliki persebaran yang normal. Rata - rata customer memberikan rating sebesar 3. Ini, menunjukkan bahwa secara umum pelanggan memberikan penilaian yang netral terhadap produk atau layanan yang diterima.

- Cost of the product memiliki rata - rata dan median yang tidak berbeda jauh, yaitu 210.2 usd dan 214 usd. Hal ini berindikasi bahwa harga produk memiliki persebaran yang cenderung normal. Harga produk termurah adalah 96 usd, sedangkan yang termahal berharga 310 usd.

- Sekitar 25% customers sudah melakukan pembelian setidaknya 3 kali. Ini menunjukkan adanya basis pelanggan yang setia dan kemungkinan adanya peluang untuk meningkatkan loyalitas pelanggan.

- Rata-rata discount yang ditawarkan sekitar 13% dan mediannya 7%. Perbedaan nilai mean dan median yang cukup signifikan tersebut mengindikasikan distribusi positively skewed. Menurut tim kami, hal ini normal karena jika diskon besar yang ditawarkan terlalu banyak akan mengurangi profit dari perusahaan. Diskon terendah yang ditawarkan adalah 1% dan terbesarnya 65%. Selain itu, strategi diskon yang bervariasi dan cenderung rendah dapat digunakan untuk menarik pelanggan tanpa mengorbankan profitabilitas perusahaan.

- Weight in gms (berat produk) memiliki perbedaan rata-rata dan median yang sangat besar, dengan meannya 3634 gram dan mediannya 4149 gram. Dapat dipastikan kolom ini memiliki distribusi skewed. Berat minimum suatu produk ialah 1001 gram dan berat maksimum suatu produk 7846 gram.

- Kolom Arrived_Late (kolom target kita) memiliki perbedaan jumlah data yang relatif kecil jika dibandingkan dengan jumlah keseluruhan data yang kita miliki, sedikit tidak seimbang.

In [ ]:
# cek data rows yang negatif apakah ada
negative = (df.select_dtypes(include=['int64','float64']) < 0 ).any(axis=1)


# Menampilkan indeks atau baris yang berisi nilai negatif
if negative.any():
    print("Baris dengan nilai negatif:")
    print(df[negative].index)
else:
    print("Tidak ada baris dengan nilai negatif")

**Data-data numerik pada dataset ini valid dan tidak ada yang aneh karena tidak ditemukan data negatif.**

In [ ]:
df[cats].describe().T

In [ ]:
# check value count data kategorik
for col in cats:
    print(f'''Value count kolom {col}:''')
    print(df[col].value_counts())
    print()

print('Value count kolom Arrived_Late:')
print(df['Arrived_Late'].value_counts())

In [ ]:
# visualization categorical variable
for i in range(0, len(cats)):
    plt.subplot(2, 3, i+1)
    sns.countplot(data=df, x=cats[i], color='navy', orient='h')
    plt.tight_layout()

- Pada kolom Warehouse_block, kategori F memiliki frekuensi tertinggi sebesar 3666 dan perbedaan dengan warehouse lain cukup signifikan, hal ini membuat ketimpangan data dan bisa menyebabkan kekurangan stock dan membebani gudang untuk penyediaan alokasi barang sesuai antrian.

- Kolom Mode_of_Shipment pun mengalami ketimpangan data paling tinggi sebesar 7462, dimana Produk yang dikirim dengan Ship jauh lebih banyak dibanding dua mode pengiriman lainnya. Kemungkinan akan terjadi keterlambatan karena antrian panjang yang tidak efisien. Ship lebih banyak di pilih karena ekonomis atau lebih cocok untuk barang-barang besar dan berat elektronik.

- Di kolom Product_importance, kategori Low dan Medium memiliki data yang cukup berimbang, masing-masing sebesar 5297 dan 4754. Namun, kategori High sangatlah sedikit. Menurut kami, hal ini wajar karena mungkin customer lebih memilih untuk bertransaksi secara langsung daripada bertransaksi online jika barang yang diinginkan sangat penting nilainya. Product low lebih banyak karena lebih terjangkau atau lebih banyaknya pilihan dalam kategorinya, sehingga permintaannya banyak.

- Value pada kolom gender seimbang yaitu F (5545) dan M (5454), ada indikasi bahwa pembelian produk sangat universal, preferensi produk yang ditawarkan relevan untuk gender Female dan Male, serta segmentasi pasarnya memang menargetkan ke dua gender.

# **Univariate Analysis**

In [ ]:
for i in range(0, len(nums)):
    plt.subplot(1, len(nums), i+1)
    sns.boxplot(y=df[nums[i]], color='navy', orient='v')
    plt.tight_layout()

Customer care calls, customer rating, cost of the product, dan Weight in gms tidak ada outlier

**Terdeteksi outlier pada feature Discount offered dan Prior purchase**, namun kami mungkin tidak akan menangani outlier tersebut karena pertimbangan berikut:

* Outlier yang ada di kolom diskon ini masih masuk akal. Data-data tersebut menjadi outlier mungkin karena memang jumlah datanya cenderung lebih sedikit dibandingkan dengan yang lain. Hal tersebut wajar saja, karena company pastinya tidak memberikan sebanyak mungkin sebanyak-banyaknya. Pemberian diskon yang lebih tinggi mungkin lebih diprioritaskan kepada pelanggan yang telah lama berlangganan.
* Outlier di kolom Prior Purchases pun masih wajar. Beberapa customer yang beli sampai sebanyak itu mungkin memang sudah begitu percaya pada e-commerce company tersebut.

In [ ]:
fig, axs = plt.subplots(2, 4,figsize=(20, 9))

sns.countplot(x = 'Customer_care_calls', data=df, ax=axs[0, 0])
axs[0, 0].grid(axis='y', linestyle='--')
axs[0, 0].set_xlabel('Customer Care Calls', fontsize=15)
axs[0, 0].set_ylabel('')

sns.countplot(x = 'Customer_rating', data=df, ax=axs[0, 1])
axs[0, 1].grid(axis='y', linestyle='--')
axs[0, 1].set_xlabel('Customer Rating', fontsize=15)
axs[0, 1].set_ylabel('')

sns.histplot(x = 'Cost_of_the_Product', data=df, ax=axs[0, 2], color='navy', kde=True)
axs[0, 2].set_xlabel('Cost of The Product', fontsize=15)
axs[0, 2].set_ylabel('')

sns.countplot(x = 'Prior_purchases', data=df, ax=axs[0, 3])
axs[0, 3].grid(axis='y', linestyle='--')
axs[0, 3].set_xlabel('Prior Purchases', fontsize=15)
axs[0, 3].set_ylabel('')

sns.histplot(x = 'Discount_offered', data=df, ax=axs[1, 0], color='navy', kde=True)
axs[1, 0].set_xlabel('Discount Offered', fontsize=15)
axs[1, 0].set_ylabel('')

sns.histplot(x = 'Weight_in_gms', data=df, ax=axs[1, 1], color='navy', kde=True)
axs[1, 1].set_xlabel('Weight(grams)', fontsize=15)
axs[1, 1].set_ylabel('')

sns.countplot(x = 'Arrived_Late', data=df, ax=axs[1, 2])
axs[1, 2].grid(axis='y', linestyle='--')
axs[1, 2].set_xlabel('Arrived Late', fontsize=15)
axs[1, 2].set_ylabel('')

axs[1, 3].axis('off')

plt.tight_layout()
plt.show();

- Customer_care_calls memiliki persebaran data yang cukup normal. Dengan menerima jumlah panggilan sebanyak 3-4 kali calls. Hal ini, mengindikasikan tingkat kepercayaan customer terhadap pelayanan pengiriman masih tergolong rendah. Perlunya peningkatan pelayanan pengiriman agar tepat waktu sehingga jumlah panggilan menjadi turun di 1-2 kali.

- Customer rating sangat balance, mengindikasikan pelayanan masih belum begitu memberikan dampak pada rating. 

- Data Cost_of_the_Product memiliki distribusi bimodal. Lonjakan biaya pada produk terjadi 2 kali, yaitu berkisar antara 180 - 250. Perlunya peningkatan control agar cost_of_product dapat lebih stabil.

- Persebaran data pada kolom Prior_purchases, grafiknya mengindikasikan kebanyakan customer baru melakukan pembelian sekitar 2-4 kali. Maka dari itu, kolom ini berisiko memiliki outlier atas. Untuk analisis lebih lanjut, lebih baik kita gunakan median dan IQR daripada mean dan standard deviation.

- Persebaran data pada kolom Discount_offered. Dapat kita lihat, bahwa sedikit sekali yang mendapatkan diskon di atas 20%. Hal tersebut normal saja karena mungkin hanya customer-customer tertentu saja yang mendapatkan penawaran diskon sebesar itu. Sama seperti kolom Prior_purchases, untuk analisis lebih lanjut, lebih baik kita gunakan median dan IQR daripada mean dan standard deviation.

- Kolom Weight_in_gms (Berat) memiliki distribusi bimodal. Terlihat pada grafik bahwa jumlah barang didominasi oleh product berukuran +-1000 gms dan +-5000 gms, dari sini bisa ditentukan prioritas mode of shipment agar pengiriman product lebih cepat dan effisien.

- Arrived_Late adalah kolom target, kolom ini merupakan data categorical yang sudah di-encode. Jadi, sebenarnya grafik di atas tidak memberikan insight yang begitu berarti. Namun, kita dapat melihat bahwa datanya imbalance, tepat waktu (0) dan tidak tepat waktu sedikit lebih banyak (1).

# Multivariate Analysis

In [ ]:
corr_matrix = df[nums].corr()
mask = np.zeros_like(corr_matrix)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(corr_matrix, annot=True,mask=mask, cmap="YlGnBu", fmt='.2f',square=True)

- Tidak ada korelasi yang kuat antara fitur-fitur dan keterlambatan pengiriman (“Arrived_Late”). Korelasi tertinggi terjadi antara Discount offered dan Arrived_Late dengan nilai 0.40 artinya semakin besar diskon yang ditawarkan maka semakin besar kemungkinan delivery-nya terlambat.
- Selain itu terdapat korelasi negatif yang relatif tinggi antara “Weight_in_gms” dan “Arrived_Late” dengan nilai -0.27, ini menunjukkan korelasi negatif yang lemah. Sehingga semakin berat paketnya, kemungkinan untuk sampai terlambat semakin kecil.
- Terdapat korelasi yang relatif tinggi antara "Cost_of_the_product" dan "Customer_care_calls" artinya semakin besar cost of the product kemungkinan semakin besar juga customer calls yang diterima..
- Customer ratings tidak memiliki korelasi dengan discount_offered dan weight_in_gms.
- Prior purchase dan Customer care calls dengan nilai 0.18, ini memiliki korelasi positif yang lemah, artinya ketika terjadi repeat pembelian, semakin banyak Customer care menerima banyak panggilan.

In [ ]:
plt.figure(figsize=(15, 15))
sns.pairplot(df, diag_kind='kde', hue='Arrived_Late', palette={0:'g', 1:'r'})

Persebaran fitur-fitur dengan target variabel (Arrived_Late) cenderung tidak berpola pasti, tapi ada beberapa pola yang unik

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(nums)): # untuk setiap kolom numerik
    ax = fig.add_subplot(3, 3, i+1) # kita set posisi catplot/stripplotnya di layout
    sns.stripplot(ax=ax, data=df, x='Warehouse_block', y=nums[i], hue='Arrived_Late', palette={0:'g', 1:'r'}) # gambar catplot/stripplotnya
    plt.legend(loc='upper right')
    plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(nums)): # untuk setiap kolom numerik
    ax = fig.add_subplot(3, 3, i+1) # kita set posisi catplot/stripplotnya di layout
    sns.stripplot(ax=ax, data=df, x='Mode_of_Shipment', y=nums[i], hue='Arrived_Late', palette={0:'g', 1:'r'}) # gambar catplot/stripplotnya
    plt.legend(loc='upper right')
    plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(nums)): # untuk setiap kolom numerik
    ax = fig.add_subplot(3, 3, i+1) # kita set posisi catplot/stripplotnya di layout
    sns.stripplot(ax=ax, data=df, x='Product_importance', y=nums[i], hue='Arrived_Late', palette={0:'g', 1:'r'}) # gambar catplot/stripplotnya
    plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(15, 15))
for i in range(0, len(nums)): # untuk setiap kolom numerik
    ax = fig.add_subplot(3, 3, i+1) # kita set posisi catplot/stripplotnya di layout
    sns.stripplot(ax=ax, data=df, x='Gender', y=nums[i], hue='Arrived_Late', palette={0:'g', 1:'r'}) # gambar catplot/stripplotnya
    plt.legend(loc='upper right')
    plt.tight_layout()

In [ ]:
df['Discount_offered_bins'] = pd.cut(df['Discount_offered'], bins=np.arange(0, 101, 10), labels=['0%-10%', '10%-20%', '20%-30%', '30%-40%', '40%-50%', '50%-60%', '60%-70%', '70%-80%', '80%-90%', '90%-100%'])
df['Weight_bins'] = pd.cut(df['Weight_in_gms'], bins=np.arange(0, 10001, 2000), labels=['0-2000', '2000-4000', '4000-6000', '6000-8000', '8000-10000'])

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

normalized_discount_offered = df.groupby('Discount_offered_bins')['Arrived_Late'].value_counts(normalize=True).mul(100).reset_index()
normalized_discount_offered.columns = ['Discount_offered', 'Arrived_Late', 'Percentage']
normalized_discount_offered = normalized_discount_offered.pivot_table(values='Percentage', columns='Arrived_Late', index='Discount_offered')
normalized_discount_offered.plot(kind='bar', stacked=True, ax=axs[0], color=['green', 'red'])

normalized_weight = df.groupby('Weight_bins')['Arrived_Late'].value_counts(normalize=True).mul(100).reset_index()
normalized_weight.columns = ['Weight_in_gms', 'Arrived_Late', 'Percentage']
normalized_weight = normalized_weight.pivot_table(values='Percentage', columns='Arrived_Late', index='Weight_in_gms')
normalized_weight.plot(kind='bar', stacked=True, ax=axs[1], color=['green', 'red'])

plt.show()

- Product yang mendapatkan diskon lebih dari 10 % tidak ada yang sampai tepat waktu. Mungkin hal ini terjadi, karena tidak begitu banyak event yang memberikan discount sehingga volume pesanan masih dalam batas wajar.
- Berdasarkan hasil analisis kami, ada hal unik yang terdapat pada kolom berat. Dimana product dengan berat di dalam range 2000-4000 gram tidak ada yang sampai tepat waktu.

Kedua insight unik yang kami dapatkan ini akan menjadi landasan bagi kami saat melakukan proses lanjutan nanti, contohnya adalah **proses Feature Engineering.** Kami akan membuat dua kolom baru, yaitu **weight_between_2kg_and_4kg** dan **is_discount_>10%**. 

In [ ]:
df[(df['Weight_in_gms']>2000) & (df['Weight_in_gms']<4000) & (df['Arrived_Late']==0)]

Berdasarkan tabel di atas, terlihat bahwa produk dengan berat 2000 - 4000 tidak ada yang mencapai tepat waktu

In [ ]:
df[(df['Discount_offered']>10) & (df['Arrived_Late']==0)]

Berdasarkan tabel di atas, dengan discount >10 tidak ada produk yang mencapai tepat waktu atau mengalami keterlambatan

# Numerical vs Target

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

normalized_ccc = df.groupby('Customer_care_calls')['Arrived_Late'].value_counts(normalize=True).mul(100).reset_index()
normalized_ccc.columns = ['Customer_care_calls', 'Arrived_Late', 'Percentage']
normalized_ccc = normalized_ccc.pivot_table(values='Percentage', columns='Arrived_Late', index='Customer_care_calls')
normalized_ccc.plot(kind='bar', stacked=True, ax=axs[0, 0], color=['green', 'red'])
axs[0, 0].set_xlabel('Customer Care Calls')

normalized_cr = df.groupby('Customer_rating')['Arrived_Late'].value_counts(normalize=True).mul(100).reset_index()
normalized_cr.columns = ['Customer_rating', 'Arrived_Late', 'Percentage']
normalized_cr = normalized_cr.pivot_table(values='Percentage', columns='Arrived_Late', index='Customer_rating')
normalized_cr.plot(kind='bar', stacked=True, ax=axs[0, 1], color=['green', 'red'])
axs[0, 1].set_xlabel('Customer Rating')

df['Cost_of_the_product_bins'] = pd.cut(df['Cost_of_the_Product'], bins=np.arange(0, 401, 100), labels=['0-100 usd', '100-200 usd', '200-300 usd', '300-400 usd'])
normalized_cost = df.groupby('Cost_of_the_product_bins')['Arrived_Late'].value_counts(normalize=True).mul(100).reset_index()
normalized_cost.columns = ['Cost_of_the_product', 'Arrived_Late', 'Percentage']
normalized_cost = normalized_cost.pivot_table(values='Percentage', columns='Arrived_Late', index='Cost_of_the_product')
normalized_cost.plot(kind='bar', stacked=True, ax=axs[1, 0], color=['green', 'red'])
axs[1, 0].set_xlabel('Product Cost')

normalized_pp = df.groupby('Prior_purchases')['Arrived_Late'].value_counts(normalize=True).mul(100).reset_index()
normalized_pp.columns = ['Prior_purchases', 'Arrived_Late', 'Percentage']
normalized_pp = normalized_pp.pivot_table(values='Percentage', columns='Arrived_Late', index='Prior_purchases')
normalized_pp.plot(kind='bar', stacked=True, ax=axs[1, 1], color=['green', 'red'])
axs[1, 1].set_xlabel('Prior Purchases')

for a in range(0, 2):
    for b in range(0, 2):
        axs[a, b].grid(axis='y', linestyle='--')
        axs[a, b].legend(['On Time', 'Late'], loc='upper right')

plt.tight_layout()
plt.show()

- Keterlambatan terjadi pada semua jumlah panggilan customer care. 
Namun seiring meningkatnya jumlah panggilan, terjadi peningkatan sedikit demi sedikit juga pada persentase ketepatan waktunya. Ada indikasi ketika ada penawaran diskon menarik, pelanggan memiliki ekspektasi yang lebih tinggi terhadap pengiriman produk yang diskon, dan jika terjadi keterlambatan, mereka lebih cenderung untuk menghubungi Customer Care.

- Semakin mahal harga suatu product, semakin besar juga kemungkinan product itu sampai tepat waktu. Ini bisa disebabkan oleh pengelolaan prioritas pengiriman, di mana produk dengan harga yang lebih tinggi, mungkin mendapatkan perhatian lebih besar dalam hal pengiriman tepat waktu.

- Prior purchases mengalami keterlambatan yang fluktuatif.

# Categorical vs Target

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

normalized_warehouse_block = df.groupby('Warehouse_block')['Arrived_Late'].value_counts(normalize=True).mul(100).reset_index()
normalized_warehouse_block.columns = ['Warehouse_block', 'Arrived_Late', 'Percentage']
normalized_warehouse_block = normalized_warehouse_block.pivot_table(values='Percentage', columns='Arrived_Late', index='Warehouse_block')
normalized_warehouse_block.plot(kind='bar', stacked=True, ax=axs[0, 0], color=['green', 'red'])
axs[0, 0].set_xlabel('Warehouse Block')

normalized_mode_shipment = df.groupby('Mode_of_Shipment')['Arrived_Late'].value_counts(normalize=True).mul(100).reset_index()
normalized_mode_shipment.columns = ['Mode_of_Shipment', 'Arrived_Late', 'Percentage']
normalized_mode_shipment = normalized_mode_shipment.pivot_table(values='Percentage', columns='Arrived_Late', index='Mode_of_Shipment')
normalized_mode_shipment.plot(kind='bar', stacked=True, ax=axs[0, 1], color=['green', 'red'])
axs[0, 1].set_xlabel('Mode of Shipment')

normalized_product_importance = df.groupby('Product_importance')['Arrived_Late'].value_counts(normalize=True).mul(100).reset_index()
normalized_product_importance.columns = ['Product_importance', 'Arrived_Late', 'Percentage']
normalized_product_importance = normalized_product_importance.pivot_table(values='Percentage', columns='Arrived_Late', index='Product_importance').reindex(['low', 'medium', 'high'])
normalized_product_importance.plot(kind='bar', stacked=True, ax=axs[1, 0], color=['green', 'red'])
axs[1, 0].set_xlabel('Product Importance')

normalized_gender = df.groupby('Gender')['Arrived_Late'].value_counts(normalize=True).mul(100).reset_index()
normalized_gender.columns = ['Gender', 'Arrived_Late', 'Percentage']
normalized_gender = normalized_gender.pivot_table(values='Percentage', columns='Arrived_Late', index='Gender')
normalized_gender.plot(kind='bar', stacked=True, ax=axs[1, 1], color=['green', 'red'])
axs[1, 1].set_xlabel('Gender')

yticks = np.arange(0, 101, 10)
label = [str(i)+'%' for i in yticks ]
for a in range(0, 2):
    for b in range(0, 2):
        axs[a, b].set_yticks(yticks, label)
        axs[a, b].grid(axis='y', linestyle='--')
        axs[a, b].legend(['On Time', 'Late'], loc='upper right')

plt.tight_layout()
plt.show()

**Hubungan antara Blok Gudang, Mode Shipment, dan Gender dengan Late Arrival:**
Tidak nampak adanya perbedaan yang signifikan dalam tingkat keterlambatan pengiriman berdasarkan blok gudang, mode pengiriman, atau jenis kelamin (gender). Ini bisa berarti bahwa faktor-faktor ini mungkin tidak memiliki pengaruh yang signifikan terhadap keterlambatan pengiriman.

**Product Importance:**
Pada produk dengan tingkat kepentingan tinggi (High), terlihat bahwa persentase keterlambatan pengiriman lebih besar dibandingkan dengan produk yang memiliki tingkat kepentingan lainnya. Namun, perbedaannya tidak dianggap signifikan.


### **Pengecekan Chi Square Contingency Variabel Independen (Categorical)**

In [ ]:
def chi_square(column):
    chgrup = df.groupby(column)['Arrived_Late'].value_counts(normalize=True).to_frame()
    chgrup.columns = ['Proportion']
    chpvt = chgrup.reset_index().pivot(columns='Arrived_Late',index=column,values='Proportion')
    display(chpvt*100)
    
    contingency = df.groupby(column)['Arrived_Late'].value_counts().to_frame().reset_index()
    contingency = contingency.pivot(columns='Arrived_Late',index=column,values='count')
    stat, p, dof, expected = chi2_contingency(contingency)
    
    if(p<0.05):
        print(f'\nKolom {column} berkorelasi kuat dengan kolom Arrived Late.\n\n')
    else:
        print(f'\nKolom {column} berkorelasi lemah dengan kolom Arrived Late.\n\n')
        
    print('------------------------------------------------------------------------------')

for cat in cats: 
    chi_square(cat)

**Dari chi square test, atribut kategorikal yang memiliki pengaruh kuat terhadap Arrived_Late adalah Product_Importance**

### **Pengecekan Variance Inflation Factor Variabel Independen (Numerical)**

In [ ]:
import statsmodels.api as sm
import pandas as pd

df.head()

In [ ]:
## Define dependent variable
y = df['Arrived_Late']

## Define Indeoendent variable
x = df[['Customer_care_calls','Customer_rating','Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']]

## Add a constant to the independent variables matrix
x = sm.add_constant(x)

## Fit the OLS model
model = sm.OLS(y, x).fit()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Get variables for which to compute VIF and add intercept term
x = sm.add_constant(df[['Customer_care_calls','Customer_rating','Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']])

## Compute and view VIF
vif = pd.DataFrame()
vif['variables'] = x.columns
vif['VIF'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]

print(vif)

**Pengecekan Variance Inflation Factor Variabel Independen (numerical) di atas:** Terlihat nilai VIF seluruh variabel bebas di atas nilainya di rentang 1, artinya tidak terdeteksi efek multikolinearitas. Dikarenakan ambang batas VIF adalah 10. Maka dalam contoh Machine Learning Supervised Regresi dapat dilakukan. Namun, untuk tim kami menggunakan Supervised Classification.

## **Additional Insight and Visualization**

In [ ]:
#melihat rata-rata diskon yang diberikan pada setiap customer rating 
series = df.groupby(['Customer_rating','Arrived_Late'])['Discount_offered'].apply(lambda x:round(x.mean(),2)).reset_index(name='Rata-rata')
series

In [ ]:
# Membuat pivot table
pivot_df = series.pivot(index='Customer_rating', columns='Arrived_Late', values='Rata-rata')

# Menampilkan pivot table
pivot_df.head()

#### **Analisis**
**Rata-rata diskon** yang diberikan pada setiap tingkat rating pelanggan **('Customer_rating')** tergantung pada apakah pengiriman tepat waktu **('Arrived_Late' = 0)** atau terlambat **('Arrived_Late' = 1):**

**Pengaruh Keterlambatan Terhadap Rata-rata Diskon:** Dapat dilihat bahwa rata-rata diskon cenderung lebih tinggi untuk pengiriman yang terlambat (Arrived_Late = 1), dibandingkan dengan pengiriman tepat waktu (Arrived_Late = 0) untuk setiap tingkat rating pelanggan. Ini bisa menunjukkan bahwa perusahaan memberikan diskon yang lebih besar sebagai insentif atau kompensasi atas keterlambatan dalam pengiriman.

**Perbedaan Rata-rata Diskon antar Tingkat Rating:** Terdapat variasi dalam rata-rata diskon antar tingkat rating pelanggan. Misalnya, pelanggan dengan rating 2 memiliki rata-rata diskon yang sedikit lebih tinggi dibandingkan pelanggan dengan rating lainnya.

**Stabilitas Rata-rata Diskon:** Dalam setiap tingkat rating, rata-rata diskon tampak cukup konsisten antara pengiriman tepat waktu dan terlambat. Ini menunjukkan bahwa strategi penentuan diskon mungkin tidak terlalu bergantung pada keterlambatan pengiriman.



#### **Kesimpulan:**
Untuk pengiriman terlambat lebih banyak mendapatkan diskon dan ini cukup berdampak pada customer rating, walaupun masih ada yang memberikan rating kurang memuaskan.

#### **Business recommendation:**
- Jika tujuannya adalah untuk mengurangi diskon yang diberikan pada pengiriman terlambat, perusahaan dapat mengevaluasi dan meningkatkan efisiensi rantai pasokan atau mengimplementasikan strategi pengiriman yang lebih andal.
- Atau justru dengan cara untuk menjaga customer rating tetap tinggi adalah dengan memberikan diskon ketika pengiriman tersebut terlambat.

In [ ]:
new_df = df[(df['Weight_in_gms']>2000) & (df['Weight_in_gms']<4000)]

In [ ]:
plt.figure(figsize=(10,7))
chart = sns.countplot(data=new_df, x='Mode_of_Shipment')

sns.set(font_scale= 1.5)
sns.set_style('whitegrid')
sns.set_palette('prism')

chart.set_xlabel("Mode of Shipment", fontsize = 15)
chart.set_ylabel("Total", fontsize = 15)

Untuk weight yang berkisar antara 2000-4000 gram, tidak ada barang yang sampai tepat waktu.
Mode ship menjadi mode yang paling tinggi penggunaannya dan menyebabkan keterlambatan akibat berat 2000-4000 gram.

In [ ]:
plt.figure(figsize=(10,7))
chart = sns.countplot(data=new_df, x='Warehouse_block')

sns.set(font_scale= 1.5)
sns.set_style('whitegrid')
sns.set_palette('prism')

chart.set_xlabel("Blok Gudang", fontsize = 15)
chart.set_ylabel("Total", fontsize = 15)

Warehouse F menjadi gudang yang paling banyak digunakan untuk weight 2000-4000 gram.

In [ ]:
pivot = pd.pivot_table(
data = new_df,
index='Mode_of_Shipment',
columns='Warehouse_block',
aggfunc={'Warehouse_block':['count']})
pivot = pivot.reset_index()
column = ['Shipment','A','B','C','D','F']
pivot.columns = column
pivot

Mode pengiriman Ship cenderung memiliki unit warehouse blok lebih banyak dibandingkan unit lainnya, sekitar >50% dibandingkan unit warehouse di Flight & Road.

In [ ]:
df_melted = pd.melt(id_vars='Shipment',frame=pivot)
df_melted

In [ ]:
plt.figure(figsize=(10,7))
chart = sns.barplot(x='variable', y='value',hue='Shipment', data=df_melted)
sns.set(font_scale= 1.5)
sns.set_style('whitegrid')
sns.color_palette('Paired')

chart.set_xlabel("Blok Gudang", fontsize = 15)
chart.set_ylabel("Total", fontsize = 15)

#### **Insight:**
Untuk weight 2000-4000 gram tidak ada pengiriman yang tepat waktu, ternyata di warehouse F dan mode ship paling banyak digunakan.

#### **Business Recommendation:**
Memberikan rekomendasi untuk melakukan pengiriman menggunakan mode road untuk jarak pendek dan flight untuk jarak jauh pada pengiriman dengan berat 2000 - 4000 dan melakukan persebaran pada masing-masing warehouse dalam menyimpan barang sehingga tidak terjadi penumpukan pada satu warehouse

### **Metrics Evaluation yang Akan Digunakan**
Dalam training machine learning model, kami memilih menggunakan **F1-Score** karena dianggap lebih seimbang dan tidak menyebabkan kebingungan pada model dalam mempelajari hasilnya.

Sementara untuk rekomendasi bisnis, kita akan menggunakan **Precision** dalam memberikan prediksi dampak bisnisnya. 

### **Business Recommendation**

* Dikarenakan terjadi imbalance warehouse, maka perlu untuk melakukan load balancing warehouse lainnya, agar tidak terjadi penumpukan barang yang menyebabkan keterlambatan pengiriman. Jika warehouse lainnya merata, ada kemungkinan penggunaan mode pengiriman road juga lebih tinggi.

* Memberikan rekomendasi alternatif mode pengiriman yaitu Road dan Flight, selain Ship kepada customer, terutama jika customer membutuhkan barang tersebut cepat sampai.

* Monitoring untuk pemberian diskon >10% agar tidak mempengaruhi volume order, tujuannya agar tidak berdampak pada keterlambatan pengiriman. 

* Memberikan diskon tambahan untuk customer, agar penilaian ratingnya meningkat. Selain itu, melakukan survei kepuasan dan mengumpulkan feedback ke customer agar mengetahui root cause kendala dapat ditangani dengan baik.

### REFERENCE 1 : https://lifewithdata.com/2023/06/04/how-to-calculate-variance-inflation-factor-vif-in-python/
### REFERENCE 2 : https://statorials.org/id/cara-menghitung-vive-dengan-python/
### REFERENCE 3 : https://www.kaggle.com/code/ridhoaryo/sba-loan-approval-model